In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["tensorwaves[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Usage

TensorWaves is a package for fitting general mathematical expressions to data distributions. The fundamentals behind the package are illustrated in {doc}`/usage/basics`.

While general in design, the package is intended for doing {doc}`Partial Wave Analysis <pwa:index>`. First, the {mod}`qrules` package determines which transitions are allowed from some initial state to a final state. The {mod}`ampform` package then formulates those transitions mathematically as an {ref}`amplitude model <usage:Construct a model>`. TensorWaves can then create a function for this expression in some computational back-end. Finally, TensorWaves can {ref}`'fit' <usage:Optimize the model>` this function to a data sample. Optionally, a data sample can be {ref}`generated <usage:Generate data sample>` from the model.

This page shows a brief overview of the complete workflow. More info about each step can be found under {ref}`usage:Step-by-step workflow`.

## Overview

In [ ]:
import logging
import warnings

import ampform
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
import qrules
import sympy as sp
from ampform.dynamics.builder import create_relativistic_breit_wigner_with_ff

from tensorwaves.data import generate_data, generate_phsp
from tensorwaves.data.transform import SympyDataTransformer
from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.optimizer import Minuit2
from tensorwaves.optimizer.callbacks import CSVSummary

logger = logging.getLogger()
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

### Construct a model

:::{seealso}

{doc}`usage/step1`

:::

:::{margin}

This example fits a simple model. See {doc}`/usage/step1` for a more complicated model.

:::

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=[
        "f(0)(980)",
        "f(0)(1500)",
        "f(0)(1710)",
    ],
    allowed_interaction_types=["strong", "EM"],
    formalism="canonical-helicity",
)
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
model_builder = ampform.get_builder(reaction)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

In [ ]:
model.components[
    R"A_{J/\psi(1S)_{-1} \xrightarrow[S=1]{L=0} f_{0}(980)_{0} \gamma_{-1};"
    R" f_{0}(980)_{0} \xrightarrow[S=0]{L=0} \pi^{0}_{0} \pi^{0}_{0}}"
].doit()

### Generate data sample

:::{seealso}

{doc}`usage/step2`

:::

In [ ]:
intensity = create_parametrized_function(
    expression=model.expression.doit(),
    parameters=model.parameter_defaults,
    backend="jax",
)

In [ ]:
helicity_transformer = SympyDataTransformer.from_sympy(
    model.kinematic_variables, backend="numpy"
)
initial_state_mass = reaction.initial_state[-1].mass
final_state_masses = {i: p.mass for i, p in reaction.final_state.items()}
phsp_momenta = generate_phsp(100_000, initial_state_mass, final_state_masses)
data_momenta = generate_data(
    10_000,
    initial_state_mass,
    final_state_masses,
    helicity_transformer,
    intensity,
)

In [ ]:
phsp = helicity_transformer(phsp_momenta)
data = helicity_transformer(data_momenta)

In [ ]:
import numpy as np
from matplotlib import cm

resonances = sorted(
    reaction.get_intermediate_particles(),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(resonances))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]


def indicate_masses():
    plt.xlabel("$m$ [GeV]")
    for i, p in enumerate(resonances):
        plt.gca().axvline(
            x=p.mass, linestyle="dotted", label=p.name, color=colors[i]
        )


fig, ax = plt.subplots(figsize=(9, 4))
ax.hist(data["m_12"], bins=100, alpha=0.5, density=True)
indicate_masses()
plt.legend()
plt.show()

### Optimize the model

:::{seealso}

{doc}`usage/step3`

:::

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def compare_model(
    variable_name,
    data,
    phsp,
    intensity_model,
    bins=150,
):
    intensities = intensity_model(phsp)
    _, ax = plt.subplots(figsize=(9, 4))
    data_1d = data[variable_name]
    ax.hist(
        data_1d,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    phsp_1d = phsp[variable_name]
    ax.hist(
        phsp_1d,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="initial fit model",
        density=True,
    )
    indicate_masses()
    ax.legend()

In [ ]:
estimator = UnbinnedNLL(
    intensity,
    data,
    phsp,
    backend="jax",
)
initial_parameters = {
    "m_f(0)(980)": 0.93,
    "m_f(0)(1500)": 1.45,
    "m_f(0)(1710)": 1.8,
    "Gamma_f(0)(980)": 0.1,
    "Gamma_f(0)(1710)": 0.2,
}
original_parameters = intensity.parameters
intensity.update_parameters(initial_parameters)

In [ ]:
compare_model("m_12", data, phsp, intensity)
print("Number of free parameters:", len(initial_parameters))

In [ ]:
callback = CSVSummary("fit_traceback.csv")
minuit2 = Minuit2(callback)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

In [ ]:
optimized_parameters = fit_result.parameter_values
intensity.update_parameters(optimized_parameters)
compare_model("m_12", data, phsp, intensity)

In [ ]:
converters = {p: lambda s: complex(s).real for p in initial_parameters}
fit_traceback = pd.read_csv("fit_traceback.csv", converters=converters)
fig, (ax1, ax2) = plt.subplots(
    2, figsize=(7, 8), gridspec_kw={"height_ratios": [1, 1.8]}
)
fit_traceback.plot("function_call", "estimator_value", ax=ax1)
fit_traceback.plot("function_call", sorted(initial_parameters), ax=ax2)
ax1.set_title("Negative log likelihood")
ax2.set_title("Parameter values")
ax1.set_xlabel("function call")
ax2.set_xlabel("function call")
fig.tight_layout()
ax1.legend().remove()
legend_texts = ax2.legend().get_texts()
for text in legend_texts:
    latex = f"${sp.latex(sp.Symbol(text.get_text()))}$"
    latex = latex.replace("\\\\", "\\")
    if latex[2] == "C":
        latex = fR"\left|{latex}\right|"
    text.set_text(latex)
for line in ax2.get_lines():
    label = line.get_label()
    color = line.get_color()
    ax2.axhline(
        y=complex(original_parameters[label]).real,
        color=color,
        alpha=0.5,
        linestyle="dotted",
    )

## Step-by-step workflow

The following pages go through the usual workflow when using {mod}`tensorwaves`. The output in each of these steps is written to disk, so that each notebook can be run separately.

```{toctree}
---
maxdepth: 2
---
usage/step1
usage/step2
usage/step3
usage/basics
usage/analytic-continuation
usage/faster-lambdify
```